In [ ]:
!pip install streamlit --upgrade

     |████████████████████████████████| 10.1 MB 5.4 MB/s 
     |████████████████████████████████| 76 kB 5.8 MB/s 
     |████████████████████████████████| 181 kB 59.9 MB/s 
     |████████████████████████████████| 111 kB 55.6 MB/s 
     |████████████████████████████████| 164 kB 57.5 MB/s 
     |████████████████████████████████| 4.3 MB 41.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 131 kB 61.0 MB/s 
     |████████████████████████████████| 428 kB 48.7 MB/s 
     |████████████████████████████████| 793 kB 55.1 MB/s 
     |████████████████████████████████| 130 kB 49.6 MB/s 
     |████████████████████████████████| 381 kB 47.5 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=608b8b8aab6f24dc30428015b0f38b1a44a2ba7821348761c479d0d7dd0eb8ee
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: 

In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=d8bdebc71ca9cc3532e765bcb636f4683ce84dd7cd979663cea8dda5a632b76e
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
# Import the librabries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
#twitter api credentials
consumerKey = ''
consumerSecret = ''
accessToken = ''
accessTokenSecret = ''
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
authenticate.set_access_token(accessToken, accessTokenSecret )
api = tweepy.API(authenticate, wait_on_rate_limit= True)
search_words = "NFT"
tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="en", tweet_mode ="extended").items(1000)

users_locs = [[tweet.full_text,tweet.user.screen_name, tweet.id ,tweet.user.location,tweet.user.created_at,tweet.source,tweet.favorite_count,tweet.retweet_count,tweet.user.verified] for tweet in tweets]
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['Tweets','user',"Tweet ID", "location","Created At","Source","Likes","Retweets","Verified"])
st.title("Explore Latest 1000 Tweets related to NFTs")
st.write(tweet_text)
# Clean the text

def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9]+','',text)#removed @mentions
  text = re.sub(r'#','',text)
  text = re.sub(r'RT[\s]+','',text)
  text = re.sub(r'https?:\/\/\S+','',text)
  text = re.sub(r':','',text)

  return text

tweet_text['Tweets']= tweet_text['Tweets'].apply(cleanTxt)
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

tweet_text['Subjectivity'] = tweet_text['Tweets'].apply(getSubjectivity)
tweet_text['Polarity'] = tweet_text['Tweets'].apply(getPolarity)
allWords = ' '.join([twts for twts in tweet_text['Tweets'] ])
wordCloud = WordCloud(width=500, height = 300, random_state = 21, max_font_size= 119).generate(allWords)
plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
st.title("Most frequent words in latest tweets")
st.write("Popular words and phrases are highlighted based on frequency and relevance in the image below .")
st.set_option('deprecation.showPyplotGlobalUse', False)
st.pyplot()

def getAnalysis(score) :
  if score<0:
    return 'Negative'
  elif score==0 :
    return 'Neutral'
  else:
    return 'Positive'

tweet_text['Analysis'] = tweet_text['Polarity'].apply(getAnalysis)
st.title("Tweets with their Subjectity,Polarity and Analysis")
st.write(tweet_text)
plt.figure(figsize= (8,6))
for i in range(0, tweet_text.shape[0]):
  plt.scatter(tweet_text['Polarity'][i],tweet_text['Subjectivity'][i],color = 'Blue')

plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
st.title("Graphical Representation on Sentiment Analysis of Tweets")
st.write("Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement.")
st.write("Subjective sentences generally refer to personal opinion, emotion or judgment whereas objective refers to factual information. Subjectivity is also a float which lies in the range of [0,1].0 for objective and 1 for subjective.")
st.set_option('deprecation.showPyplotGlobalUse', False)
st.pyplot()
tweet_text['Analysis'].value_counts()

plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
tweet_text['Analysis'].value_counts().plot(kind = 'bar')
st.write("Graph for comparison for number of positive, negative and neutral latest tweets.")
st.set_option('deprecation.showPyplotGlobalUse', False)
st.pyplot()
tweet_text["location"].value_counts()

plt.title('NFT tweet location analysis')
plt.xlabel('Location')
plt.ylabel('Counts')
tweet_text["location"].value_counts().nlargest(20).plot(kind = 'bar')
st.title("Graph for location of latest tweets")
st.set_option('deprecation.showPyplotGlobalUse', False)
st.pyplot()
tweet_text["location"].groupby(tweet_text["location"]).value_counts().nlargest(20)
tweet_text["Source"].value_counts()
st.title("Graph for Soure of latest tweets.")
plt.title('NFT tweet source analysis')
plt.xlabel('Source')
plt.ylabel('Counts')
tweet_text["Source"].value_counts().nlargest(5).plot(kind = 'bar')
st.set_option('deprecation.showPyplotGlobalUse', False)
st.pyplot()

Writing app.py


In [ ]:
!ls

app.py	sample_data


In [ ]:
!ngrok authtoken #enter your ngrok authtoken here

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [ ]:
from pyngrok import ngrok

In [ ]:
publ_url = ngrok.connect(port='80')
print(publ_url)
!streamlit run --server.port 80 app.py >/dev/null

NgrokTunnel: "http://1f11-34-134-173-73.ngrok.io" -> "http://localhost:80"
2022-04-29 10:48:41.751 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
